# Reading and Writing Audio Files with wave

[back to overview page](https://github.com/mgeier/python-audio/blob/master/audio-files)

The `wave` module is part of the Python standard library.

Documentation:

* http://docs.python.org/2/library/wave.html (Python 2.x)
* http://docs.python.org/3/library/wave.html (Python 3.x)

Audio data is handled with Python strings/buffers.

Advantages:

* part of the standard library, no further dependencies
* 24-bit files can be used (with manual conversion)
* partial reading is possible
* works with both Python 2 and 3

Disadvantages:

* 32-bit float not supported
* WAVEX doesn't work
* manual de-interleaving and conversion is necessary

## Reading

Reading a 16-bit WAV file is not hard, but it requires a few lines of code:

In [ ]:
import numpy as np
import wave
from contextlib import closing
from utility import pcm2float

with closing(wave.open('data/test_wav_pcm16.wav')) as w:
    nchannels = w.getnchannels()
    assert w.getsampwidth() == 2
    data = w.readframes(w.getnframes())

sig = np.frombuffer(data, dtype='<i2').reshape(-1, nchannels)

normalized = pcm2float(sig, np.float32)

But not so fast! Let's do that step-by-step and put a few explanations in between.

First, let's enable inline plotting and load all the NumPy goodies into the local namespace:

In [ ]:
%pylab inline

The only module we really need to load is `wave`. However, let's also load `contextlib.closing` in order to be able to automagically close all our opened files:

In [ ]:
import wave
from contextlib import closing

# Note: in Python 3.4, wave.open() will return a context manager, closing() won't be necessary anymore

Now we open a 16-bit WAV file, show a few informations and read its contents.

In [ ]:
with closing(wave.open('data/test_wav_pcm16.wav')) as w:
    framerate = w.getframerate()
    nframes = w.getnframes()
    nchannels = w.getnchannels()
    width = w.getsampwidth()
    print("sampling rate = {framerate} Hz, length = {nframes} samples, channels = {nchannels}, sample width = {width} bytes".format(**locals()))
    
    data = w.readframes(nframes)

We see that the *sample width* is 2 bytes, which we expected for a 16-bit file.

If the audio file has 7 channels, is 15 samples long and uses 2 bytes (16 bit) per sample, the buffer has a size of $7 \times 15 \times 2 = 210$ bytes.

In [ ]:
len(data)

Audio data is stored in a Python buffer object (basically a contiguous memory area with bytes of data). In Python 2 this looks somehow like a string object, in Python 3 it's an object of type `bytes`.

In [ ]:
type(data)

The buffer data looks like this (which isn't really helpful):

In [ ]:
data

We could convert the bytes by means of the `struct` module, but it's easier with the `frombuffer()` function from NumPy.
We have to specify (similar to how it's done in the `struct` module) how the bytes should be interpreted to yield the desired numbers.

If we would interpret them as single (unsigned) bytes, it would look like this (still not very useful):

In [ ]:
frombuffer(data, dtype='B')

To be able to do something useful with the data, we have to take pairs of 2 bytes and convert the to 16-bit integers. Thereby we have to consider that data in WAV files is stored in *little endian* format (the least significan byte comes first and then the most significant byte), which is specified with `'<'` in the format string.

Let's also reshape the array to get a column for each channel:

In [ ]:
sig = frombuffer(data, dtype='<i2').reshape(-1, nchannels)
sig

Note that neither `frombuffer()` nor `reshape()` made a copy of the data. We are still using the buffer of bytes we got from `readframes()`:

In [ ]:
sig.base.base is data

With the `flags` attribute we get a few details about the buffer. `C_CONTIGUOUS` means that the rows are contiguous (in row-major format, like it's used in C). We also see that `sig` doesn't "own" the data and that it's not writable:

In [ ]:
sig.flags

We've already got the correct values but if we want to do some signal processing with the data, it's normally easier to convert the numers to floating point format and to normalize them to a range from -1 to 1.

To do that, I wrote a little helper function called `pcm2float()`, located in the file [`utility.py`](https://github.com/mgeier/python-audio/blob/master/audio-files/utility.py), let's load it and apply it to our signal:

In [ ]:
from utility import pcm2float

normalized = pcm2float(sig, float32)

plot(normalized)
show()

Because we change the data type from int16 to float32, a copy of the array is created, which now is writable and "owns" its data:

In [ ]:
normalized.flags

24-bit files can be opened with `wave.open()` as well, but the conversion is a little more complicated. (See also http://stackoverflow.com/a/17443437/500098)

In [ ]:
with closing(wave.open('data/test_wav_pcm24.wav')) as w:
    framerate = w.getframerate()
    nframes = w.getnframes()
    nchannels = w.getnchannels()
    width = w.getsampwidth()
    print("sampling rate = {framerate} Hz, length = {nframes} samples, channels = {nchannels}, sample width = {width} bytes".format(**locals()))
    
    data = w.readframes(nframes)

Note that the sample width is 3 bytes (because we loaded a 24-bit file). Sadly, there is no 3-byte integer type in NumPy. Therefore, we have to fill each 3-byte number with a further byte and then convert it to a 4-byte integer.
We can add this byte (filled with zeros) either as most significant or a least significant byte. This doesn't change the precision of the data, we just have to remember which one it was when we do calculations with the stored values.
If we add the zero-byte as LSB, the resulting values will have the full range of a 4-byte integer, therefore we can use the `pcm2float()` function from above.
If we would add the zero-byte as MSB, the range would be limited to a 3-byte integer and we would have to write a new function for normalization.

In [ ]:
assert width == 3

temp = bytearray()

for i in range(0, len(data), 3):
    temp.append(0)
    temp.extend(data[i:i+3])

frombuffer(temp, dtype=uint8)

Now we have a bytearray where each group of 4 bytes represents an integer (in little-endian order).

Next, let's convert it to actual integers and reshape the channels into columns.

In [ ]:
sig = frombuffer(temp, dtype='<i4').reshape(-1, nchannels)

Let's put this into a function, just in case we find ourselves needing to load 24-bit WAV files some time in the future.

In [ ]:
def pcm24to32_bytearray(data, nchannels=1):
    assert len(data) % 3 == 0

    size = len(data) // 3

    temp = bytearray(size * 4)

    for i in range(size):
        newidx = i * 4 + 1
        oldidx = i * 3
        temp[newidx:newidx+3] = data[oldidx:oldidx+3]

    return np.frombuffer(temp, dtype='<i4').reshape(-1, nchannels)

This looks a bit clumsy, let's try it another way:

In [ ]:
def pcm24to32(data, nchannels=1):
    assert len(data) % 3 == 0
    temp = np.zeros((len(data) // 3, 4), dtype='b')
    
    temp[:, 1:] = np.frombuffer(data, dtype='b').reshape(-1, 3)

    return temp.view('<i4').reshape(-1, nchannels)

TODO: timing measurements!

In [ ]:
sig = pcm24to32(data, nchannels)

Finally, we normalize the values to a range from -1 to 1 (see above).

In [ ]:
normalized = pcm2float(sig, float32)

plot(normalized)
show()

WAVEX doesn't work:

In [ ]:
try:
    w = wave.open('data/test_wavex_pcm16.wav')
except:
    import traceback
    traceback.print_exc()
else:
    print("It works (unexpectedly)!")

Let's try 32-bit float:

In [ ]:
try:
    w = wave.open('data/test_wav_float32.wav')
except:
    import traceback
    traceback.print_exc()
else:
    print("It works (unexpectedly)!")

## Writing

TODO

Another way (without NumPy): http://soledadpenades.com/2009/10/29/fastest-way-to-generate-wav-files-in-python-using-the-wave-module/



## Version Info

In [ ]:
import sys, IPython
print("Versions: NumPy = {}; IPython = {}".format(numpy.__version__, IPython.__version__))

print("Python interpreter:")
print(sys.version)